# Expectation Value Cost

In [1]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random

import pickle
from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from cost_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
#np.set_printoptions(threshold=sys.maxsize)

In [2]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
X, _, _ = generate_ginibre(d, d)
U = generate_unitary(X)

kraus_target = KrausMap(U, c = 0.5, d = d, rank = 3)
choi_target = maps_to_choi([kraus_target])

num_samples = 1000
input_list = []
target_list = []
for i in range(num_samples):
    config = np.random.randint(6, size=n)
    state = prepare_input(config)
    config = np.random.randint(4, size=n)
    observable = pauli_observable(config)
    input_list.append([state, observable])
    
    state = kraus_target.apply_map(state)
    E = expectation_value(state, observable)
    
    target_list.append(E)

In [3]:
kraus_model = KrausMap(U, c = 0.1, d = d, rank = 2)

model = ModelQuantumMap(q_map = kraus_model,
                        cost = expectation_value_loss,
                        input_list = input_list,
                        target_list = target_list,
                        lr = 0.05, 
                        h = 1e-3)

model.train(num_iter = 500, 
            use_adam = True,
            N = 50)

  0%|          | 0/500 [00:00<?, ?it/s]

0: fid: 0.077, c: 0.105
1: fid: 0.074, c: 0.109
2: fid: 0.072, c: 0.114
3: fid: 0.070, c: 0.120
4: fid: 0.068, c: 0.125
5: fid: 0.066, c: 0.130
6: fid: 0.064, c: 0.136
7: fid: 0.062, c: 0.142
8: fid: 0.060, c: 0.148
9: fid: 0.058, c: 0.155
10: fid: 0.056, c: 0.162


KeyboardInterrupt: 